# Importando Bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pickle
import warnings
import os

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, classification_report
from sklearn.exceptions import ConvergenceWarning, UndefinedMetricWarning

import mnist_loader

In [ ]:
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# Carregando o MNIST dataset

In [ ]:
# Loading MNIST dataset
training_data, validation_data, test_data = mnist_loader.load_data()

X_train, Y_train = np.concatenate((training_data[0], validation_data[0])), np.concatenate((training_data[1], validation_data[1]))
X_test, Y_test = test_data[0], test_data[1]

# Definindo os parâmetros da MLP

Os parâmetros testados em busca do melhor desempenho estão descritos abaixo:
<br>
- Número de camadas: Definido aleatoriamente para cada experimento, de forma que cada MLP testado terá 1 ou mais camadas ocultas, com cada camada sendo adicionada caso o número aleatório retornado pelo random.uniform seja maior do que 0.4
- Número de neurônios: Também definido aleatoriamente para cada experimento e camada, sendo um número entre 2 e 100
- Taxa de aprendizagem: Número definido de forma aleatória entre 0.001 e 0.1, tendo no máximo 4 dígitos
- Função de ativação: Escolhido aleatoriamente, podendo ser 'identity', 'logistic', 'tanh' ou 'relu'
- Algoritmo de aprendizagem: Também escolhido aleatoriamente, podendo ser 'adam' (Adaptive Moment Estimation), 'sgd' (Stochastic Gradient Descent) ou 'lbfgs' (Limited-memory Broyden-Fletcher-Goldfarb-Shanno)

In [ ]:
# Definir a quantidade de camadas escondidas
def random_hidden_layers():
    hidden_layer = [random.randint(2, 100)]
    while random.uniform(0.0, 1.0) > 0.4:
        hidden_layer.append(random.randint(2, 100))
    return hidden_layer

def random_params_mlp(verbose=False, num_epochs=100, tol=0.0001):
    return MLPClassifier(
        activation=random.choice(['identity', 'logistic', 'tanh', 'relu']),
        batch_size='auto',
        early_stopping=True,
        hidden_layer_sizes=random_hidden_layers(),
        learning_rate_init=round(random.uniform(0.0001, 0.1), 4),
        max_iter=num_epochs,
        n_iter_no_change=3,
        solver=random.choice(['adam', 'sgd', 'lbfgs']),
        tol=tol,
        verbose=verbose,
        warm_start=True
    )

# Fazendo um "GridSearch" sobre a camada escondida

Primeiro vamos definir a estrutura do dataframe para armazenar os resultados obtidos durante o gridsearch

In [ ]:
if (os.path.exists('model_metrics.csv')):
    model_df = pd.read_csv('model_metrics.csv')
else:
    model_data = {
        'solver': [],
        'activation_function': [],
        'hidden_layers': [],
        'learning_rate': [],
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1_score': [],
        'fit_time': [],
        'total_epochs': [],
    }
    model_df = pd.DataFrame(model_data)

Em seguida, realizamos 10 experimentos, cada um com parâmetros diferentes escolhidos aleatoriamente usando a biblioteca random, que é nativa do python. Para cada experimento, iremos exibir a acurácia total e tempo de treinamento do modelo testado, além de salvar seus resultados (acurácia total, precisão total, recall total, tempo de treinamento e f1 score) e parâmetros (função de ativação, algoritmo de aprendizagem, número de epochs, número de camadas e neurônios e taxa de aprendizagem).
<br>No final de cada experimento, são salvos os três modelos com a maior acurácia total, os quais são exibidos após a conclusão de todos os 10 testes.

In [ ]:
trained_models = []

print("Activation | Solver | LR | Hidden Layers | Total Time | Total Epochs -> Accuracy")
for _ in range(10):
    mlp = random_params_mlp(num_epochs=250)

    start = time.time()
    mlp.fit(X_train, Y_train)
    end = time.time()
    total_time = round(end - start, 1)

    mlp_predictions = mlp.predict(X_test)
    accuracy = accuracy_score(Y_test, mlp_predictions)
    precision = precision_score(Y_test, mlp_predictions, average='macro')
    recall = recall_score(Y_test, mlp_predictions, average='macro')
    f1 = f1_score(Y_test, mlp_predictions, average='macro')
    print(f"{mlp.get_params()['activation']} | {mlp.get_params()['solver']} | {mlp.get_params()['learning_rate_init']} \
| {mlp.get_params()['hidden_layer_sizes']} | {total_time}s | {mlp.n_iter_} -> {accuracy}")

    trained_models.append((mlp, accuracy))
    trained_models = sorted(trained_models, key=lambda tup: tup[1], reverse=True)[:3]

    # Adicionando o modelo ao dataframe de modelos
    model_df.loc[len(model_df)] = {
        'solver': mlp.get_params()['solver'],
        'activation_function': mlp.get_params()['activation'],
        'hidden_layers': str(mlp.get_params()['hidden_layer_sizes']),
        'learning_rate': mlp.get_params()['learning_rate_init'],
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'fit_time': total_time,
        'total_epochs': mlp.n_iter_,
    }

    model_df.to_csv('model_metrics.csv', index=False)

print("\nTop 3 Models:")
for model, acc in trained_models:
    print(f"Accuracy: {acc}, Params: {model.get_params()['hidden_layer_sizes']}")

In [ ]:
model_df

In [ ]:
model_df.to_csv('model_metrics.csv', index=False)

Acima temos os parâmetros para cada um dos 25 Multilayer Perceptrons testados, junto com sua acurácia total e o tempo que levou para treinar cada um deles. <br>
Nota-se que a maioria dos modelos apresentam uma acurácia parecida, com excessão de alguns modelos que possuem 4 camadas ocultas, com o restante dos modelos apresentando uma acurácia total maior do que 0.89.

# Análise da Performance Sobre o Conjunto de Teste

In [ ]:
best_model = trained_models[0][0]

In [ ]:
mlp_predictions = best_model.predict(X_test)

## Calculando Acurácia, Precisão e Recall

Feita a previsão do modelo com os dados do conjunto de teste, foi calculado a acurácia, precisão, recall total e f1 score  usando as funções accuracy_score, precision_score, recall_score e f1_score, todos da biblioteca scikit-learn.

In [ ]:
accuracy = accuracy_score(Y_test, mlp_predictions)
precision = precision_score(Y_test, mlp_predictions, average='macro')
recall = recall_score(Y_test, mlp_predictions, average='macro')
f1 = f1_score(Y_test, mlp_predictions, average='macro')

In [ ]:
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Além disso, exibimos as principais métricas de classificação (acurácia, precisão, recall e f1 score) para cada classe para que possamos avaliar a confiabilidade do modelo na classificação de cada dígito.

In [ ]:
#Pegando o report de cada classe 
report = classification_report(Y_test, mlp_predictions, target_names=['0','1','2','3','4','5','6','7','8','9'], output_dict = True)
conf_matrix = confusion_matrix(Y_test, mlp_predictions )
# Calcular a acurácia para cada classe
accuracy_per_class = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
for i in range(10):
  report[str(i)]['accuracy'] = accuracy_per_class[i]

#Printando a tabela
df = pd.DataFrame(report)
df_inverted = df.transpose()
df = df.drop(columns=['accuracy', 'macro avg', 'weighted avg'])
df

## Plotando a Matrix de Confusão

Também exibimos a matriz de confusão abaixo, fornecendo uma outra visão da performance do melhor modelo para cada uma das suas 10 classes, de forma a avaliarmos a quantidade de instâncias de cada classe que o modelo classificou corretamente.

In [ ]:
# Confusion Matrix
conf_matrix = confusion_matrix(Y_test, mlp_predictions)
plt.figure(figsize=(8, 7))

# Heatmap
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False, xticklabels=range(10), yticklabels=range(10))

# Labels
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')

for i in range(11):
    plt.hlines(i, xmin=0, xmax=10, colors='black', linestyles='solid', linewidth=1)
    plt.vlines(i, ymin=0, ymax=10, colors='black', linestyles='solid', linewidth=1)

plt.show()

Observa-se que o modelo tem uma performance boa em todas as classes, acertando a maioria dos casos de teste, errando principalmente os dígitos 8 e 9, onde mais de 30 instâncias foram classificadas incorretamente, porém com mais de 900 instâncias, para cada um desses dígitos, sendo classificadas corretamente.

## Média e Desvio Padrão das Métricas

Por último, iremos calcular a média e desvio padrão dos 3 melhores conjuntos de parâmetros para ter um melhor entendimento do quão bom é a performance dos modelos treinados com esses parâmetros.

In [ ]:
data = {
    'Params': [],
    'Mean Accuracy': [],
    'Std Accuracy': [],
    'Mean Precision': [],
    'Std Precision': [],
    'Mean Recall': [],
    'Std Recall': [],
    'Mean Time': [],
    'Std Time': [],
}

In [ ]:
# Criando 10 modelos para as 3 melhores combinações de parâmetros
for i, model in enumerate(trained_models):
    model = model[0]
    accuracy = []
    precision = []
    recall = []
    fit_time = []
    for _ in range(10):
        start = time.time()
        model.fit(X_train, Y_train)
        end = time.time()
        fit_time.append(round(end - start, 1))

        model_predictions = model.predict(X_test)
        accuracy.append(accuracy_score(Y_test, model_predictions))
        precision.append(precision_score(Y_test, model_predictions, average='weighted'))
        recall.append(recall_score(Y_test, model_predictions, average='weighted'))

    data['Params'].append(f"C{i}")
    data['Mean Accuracy'].append(np.mean(accuracy))
    data['Std Accuracy'].append(np.std(accuracy))
    data['Mean Precision'].append(np.mean(precision))
    data['Std Precision'].append(np.std(precision))
    data['Mean Recall'].append(np.mean(recall))
    data['Std Recall'].append(np.std(recall))
    data['Mean Time'].append(np.mean(fit_time))
    data['Std Time'].append(np.std(fit_time))
df = pd.DataFrame(data)

In [ ]:
df

# Salvando o melhor modelo treinado

Nessa secção está o código para salvar o modelo de melhor performance usando o pickle, outra biblioteca nativa do python.
<br>Salvamos esse modelo em um arquivo .pkl que pode ser aberto posteriormente com o pickle.load(open(caminho_arquivo_pkl, 'rb')).

In [ ]:
filename = 'test_model.plk'

In [ ]:
# Salvando o modelo em um arquivo
with open(filename, 'wb') as f:
    pickle.dump(best_model, f)

In [ ]:
# # Carregando um modelo salvo
# with open(filename, 'rb') as f:
#     best_model = pickle.load(f)